In [1]:
from app import db
import datetime as dt
from app.core import Timeframe


start_at = dt.datetime(2022, 10, 25)
end_at = dt.datetime(2022, 11, 1)
asset = 'BTCUSD'
timeframe = Timeframe.M5

await db.start()

In [2]:
from app import indicators
import pandas as pd


candles = await db.get_candles(asset, timeframe, start_at, end_at)
print('Total candles:', len(candles))


ema20 = indicators.EMA(20)
ema_out = []


for candle in candles:
    ema_out.append({
        'open_time': candle['open_time'],
        'ema20': ema20.calculate(candle['close'])
    })
    
    
ema_df = pd.DataFrame.from_dict(ema_out)

Total candles: 2016


In [3]:
from app import charts
import altair as alt


candles_chart = charts.candles_chart(candles)
ema_chart = alt.Chart(ema_df).mark_line().encode(x='open_time', y='ema20')

candles_chart + ema_chart

alt.LayerChart(...)

In [4]:
import random
from app.core import TradeDirection


def build_random_trades_dataset():
    dset = []
    step_counter = 5

    for candle in candles:
        step_counter -= 1

        if step_counter != 0:
            continue

        step_counter = random.randint(10, 40)
        direction = random.choice((TradeDirection.BULL, TradeDirection.BEAR))

        dset.append({'time': candle['open_time'], 'price': candle['close'], 'direction': direction})
        
    return dset
        
        
trades_chart = charts.trades_chart(build_random_trades_dataset())

candles_chart + trades_chart

alt.LayerChart(...)

In [5]:
import random
from drafts_.charts import get_extended_equity_chart


def build_random_equity_dataset():
    equity = 1000.0
    now = start_at
    
    dset = [{'Time': start_at, 'Equity': equity, 'Leverage Used': 0, 'Drawdown': 0}]
    
    while now < end_at:
        now += dt.timedelta(minutes=5) * random.randint(5, 20)
        equity += random.randint(-50, 100)
        leverage = random.randint(75, 150) / 100
        
        dset.append({'Time': now, 'Equity': equity, 'Leverage Used': leverage, 'Drawdown': 0})
        
    return dset
        
        
equity_df = pd.DataFrame.from_dict(build_random_equity_dataset())

get_extended_equity_chart(equity_df)

alt.VConcatChart(...)